In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
from faker import Faker

# Initialize Faker
fake = Faker()

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Define dataset sizes
NUM_HOSPITALS = 50
NUM_PATIENTS = 5000
NUM_DOCTORS = 200
NUM_MEDICATIONS = 300
NUM_PROCEDURES = 200
NUM_INSURANCE_PROVIDERS = 20
NUM_PRESCRIPTIONS = 20000
NUM_VISITS = 30000
NUM_JULIAN_DATES = 365 * 5  # 5 years of dates

# 1. Generate HOSPITAL_DIM Data
hospitals = []
for i in range(1, NUM_HOSPITALS + 1):
    hospitals.append({
        "HOSPITAL_ID": i,
        "HOSPITAL_NAME": fake.company(),
        "STATE": fake.state(),
        "CITY": fake.city(),
        "ZIP": fake.zipcode(),
        "PHONE_NO": f"{random.randint(100,999)}-{random.randint(100,999)}-{random.randint(1000,9999)}",
        "EMERGENCY_SERV": random.choice([0, 1])  # 1 = Yes, 0 = No
    })
df_hospitals = pd.DataFrame(hospitals)
df_hospitals.to_csv("hospitals.csv", index=False)

# 2. Generate PATIENT_DIM Data
patients = []
for i in range(1, NUM_PATIENTS + 1):
    patients.append({
        "PATIENT_ID": i,
        "FIRST_NAME": fake.first_name(),
        "LAST_NAME": fake.last_name(),
        "DOB": fake.date_of_birth(minimum_age=18, maximum_age=90).strftime("%Y%m%d"),
        "AGE": random.randint(18, 90),
        "SEX": random.choice(["Male", "Female"]),
        "PRIMARY_INSUR_PROV_ID": random.randint(1, NUM_INSURANCE_PROVIDERS),
        "SECONDARY_INSUR_PROV_ID": random.randint(1, NUM_INSURANCE_PROVIDERS) if random.random() > 0.5 else None,
        "BLOOD_TYPE": random.choice(["A+", "A-", "B+", "B-", "O+", "O-", "AB+", "AB-"])
      })
df_patients = pd.DataFrame(patients)
df_patients.to_csv("patients.csv", index=False)

# 3. Generate DOCTOR_DIM Data
doctors = []
for i in range(1, NUM_DOCTORS + 1):
    doctors.append({
        "DOCTOR_ID": i,
        "FIRST_NAME": fake.first_name(),
        "LAST_NAME": fake.last_name(),
        "HOSPITAL_AFFI": random.randint(1, NUM_HOSPITALS),
        "SPECIALTY": random.choice(["Cardiology", "Pediatrics", "Orthopedics", "Neurology", "General Surgery"]),
        "PHONE_NO": f"{random.randint(100,999)}-{random.randint(100,999)}-{random.randint(1000,9999)}"
    })
df_doctors = pd.DataFrame(doctors)
df_doctors.to_csv("doctors.csv", index=False)

# 4. Generate MEDICATION_DIM Data with Better Names
med_prefixes = ["Pro", "Vent", "Corti", "Aspi", "Neuro", "Dolo", "Flu", "Amoxi", "Cipro", "Metro"]
med_suffixes = ["dal", "zine", "xil", "pan", "vir", "cort", "mycin", "mab", "pril", "lol"]

medications = []
for i in range(1, NUM_MEDICATIONS + 1):
    medications.append({
        "MEDICATION_ID": i,
        "MEDICATION_NAME": f"{random.choice(med_prefixes)}{random.choice(med_suffixes)} {random.choice(['XR', '500', 'Plus', 'Max', 'Forte', 'EC', ''] )}".strip(),
        "PHARMA_COMPANY": fake.company(),
        "CATEGORY": random.choice(["Antibiotic", "Painkiller", "Antidepressant", "Anti-inflammatory", "Cardiovascular"])
    })
df_medications = pd.DataFrame(medications)
df_medications.to_csv("medications.csv", index=False)

# 5. Generate PROCEDURE_DIM Data
procedures = []
for i in range(1, NUM_PROCEDURES + 1):
    procedures.append({
        "PROCEDURE_ID": i,
        "PROCEDURE_NAME": fake.catch_phrase(),
        "MEDICAL_CATEGORY": random.choice(["Surgery", "Diagnostic", "Therapeutic"]),
        "COST": round(random.uniform(500, 15000), 2)
    })
df_procedures = pd.DataFrame(procedures)
df_procedures.to_csv("procedures.csv", index=False)

# 6. Generate INSURANCE_PROVIDER_DIM Data
insurance_providers = []
for i in range(1, NUM_INSURANCE_PROVIDERS + 1):
    insurance_providers.append({
        "INSURANCE_PROVIDER_ID": i,
        "PROVIDER_NAME": fake.company(),
        "PLAN_TYPE": random.choice(["Basic", "Premium", "Gold", "Platinum"]),
        "PHONE_NO": f"{random.randint(100,999)}-{random.randint(100,999)}-{random.randint(1000,9999)}"
    })
df_insurance_providers = pd.DataFrame(insurance_providers)
df_insurance_providers.to_csv("insurance_providers.csv", index=False)

# 7. Generate JULIAN_DATE_DIM Data (Updated for a Broad Date Range)
start_date = pd.to_datetime("1930-01-01")
end_date = pd.to_datetime("2030-12-31")
date_range = pd.date_range(start=start_date, end=end_date)

julian_dates = []
for date in date_range:
    julian_dates.append({
        "JULIAN_DAY": date.strftime("%Y%m%d"),  # Format as YYYYMMDD (e.g., 20240303)
        "ACTUAL_DT": date.strftime("%Y-%m-%d"),
        "DAY_NAME": date.strftime("%A"),
        "DAY_ABBREV": date.strftime("%a"),
        "DAY_IN_YEAR": date.timetuple().tm_yday,
        "DAY_IN_MONTH": date.day,
        "DAY_IN_WEEK": date.weekday(),
        "MONTH_NAME": date.strftime("%B"),
        "MONTH_ABBREV": date.strftime("%b"),
        "MONTH_NUM": date.month,
        "YEAR_NAME": str(date.year),
        "YEAR_NUM": date.year,
        "QUARTER": (date.month - 1) // 3 + 1
    })
df_julian_dates = pd.DataFrame(julian_dates)
df_julian_dates.to_csv("julian_dates.csv", index=False)

# 8. Generate HOSPITAL_VISIT_FACT Data (Updated)
visits = []
for i in range(1, NUM_VISITS + 1):
    admission_date = fake.date_between(start_date="-2y", end_date="today")
    discharge_date = admission_date + pd.Timedelta(days=random.randint(1, 15))  # Stay duration randomized

    visits.append({
        "VISIT_ID": i,
        "ADMISSION_DATE": admission_date.strftime("%Y%m%d"),  # Now in Julian date format
        "DISCHARGE_DATE": discharge_date.strftime("%Y%m%d"),    # Now in Julian date format
        "PATIENT_ID": random.randint(1, NUM_PATIENTS),
        "DOCTOR_ID": random.randint(1, NUM_DOCTORS),
        "HOSPITAL_ID": random.randint(1, NUM_HOSPITALS),
        "INSURANCE_PROVIDER_ID": random.randint(1, NUM_INSURANCE_PROVIDERS),
        "ROOM_NO": random.randint(100, 999),
        "ADMISSION_TYPE": random.choice(["Emergency", "Routine", "Urgent"]),
        "PROCEDURE_ID": random.randint(1, NUM_PROCEDURES),
        "DIAGNOSIS": random.choice(["Flu", "Pneumonia", "Fracture", "Diabetes", "Hypertension"]),
        "BILLING_AMOUNT": round(random.uniform(500, 15000), 2)
    })
df_visits = pd.DataFrame(visits)
df_visits.to_csv("hospital_visits.csv", index=False)

# 9. Generate PRESCRIPTION_FACT Data
prescriptions = []
for i in range(1, NUM_PRESCRIPTIONS + 1):
    prescriptions.append({
        "PRESCRIPTION_ID": i,
        "PATIENT_ID": random.randint(1, NUM_PATIENTS),
        "DOCTOR_ID": random.randint(1, NUM_DOCTORS),
        "MEDICATION_ID": random.randint(1, NUM_MEDICATIONS),
        "QUANTITY": random.randint(1, 30),
        "DOSAGE": f"{random.randint(5, 500)}mg",
        "FREQUENCY": random.choice(["Once a day", "Twice a day", "Every 6 hours"]),
        "PRESCRIBED_DATE": fake.date_between(start_date="-2y", end_date="today").strftime("%Y%m%d"),
        "REFILL_ALLOWED": random.choice([0, 1]),
        "REFILL_COUNT": random.randint(0, 5)
    })
df_prescriptions = pd.DataFrame(prescriptions)
df_prescriptions.to_csv("prescriptions.csv", index=False)

'\n# 9. Generate PRESCRIPTION_FACT Data\nprescriptions = []\nfor i in range(1, NUM_PRESCRIPTIONS + 1):\n    prescriptions.append({\n        "PRESCRIPTION_ID": i,\n        "PATIENT_ID": random.randint(1, NUM_PATIENTS),\n        "DOCTOR_ID": random.randint(1, NUM_DOCTORS),\n        "MEDICATION_ID": random.randint(1, NUM_MEDICATIONS),\n        "QUANTITY": random.randint(1, 30),\n        "DOSAGE": f"{random.randint(5, 500)}mg",\n        "FREQUENCY": random.choice(["Once a day", "Twice a day", "Every 6 hours"]),\n        "PRESCRIBED_DATE": fake.date_between(start_date="-2y", end_date="today").strftime("%Y%m%d"),\n        "REFILL_ALLOWED": random.choice([0, 1]),\n        "REFILL_COUNT": random.randint(0, 5)\n    })\ndf_prescriptions = pd.DataFrame(prescriptions)\ndf_prescriptions.to_csv("prescriptions.csv", index=False)\n'